# Previsión del precio de la electricidad con redes neuronales

### 💡 Objetivo:

Este cuaderno busca utilizar técnicas avanzadas de aprendizaje automático para **prever el precio de la electricidad**. Mediante el uso de redes neuronales, como LSTM, CNN y RNN, se intentará modelar y entender las complejidades y patrones inherentes a los datos de series temporales.

### ⚡ Importancia de Prever el Precio de la Electricidad:

La electricidad es un recurso esencial en la sociedad moderna y su **precio** tiene un impacto directo en la economía y en la vida cotidiana. **Prever su precio** permite a las empresas y consumidores planificar sus gastos y promueve la **estabilidad del mercado energético**. Además, con la incorporación de **fuentes de energía renovable**, la **previsión precisa del precio** se vuelve crucial.

###  📜 Antecedentes:

La electricidad, fundamental para el progreso social, ha pasado a ser un bien comerciable en países desregulados. La **Ley del Sector Eléctrico 54/1997 en España** introdujo incertidumbre en la industria eléctrica, ya que los **precios de la electricidad** están influenciados por innumerables factores y la dificultad de **almacenar electricidad**. La **creciente demanda** de métodos de pronóstico fiables ha abierto una vía para la **investigación**.

El objetivo de este análisis es **predecir el precio de la electricidad de la hora** siguiente utilizando datos históricos del precio de la electricidad, así como otras variables relevantes relacionadas con la generación de energía y las condiciones meteorológicas. Esta investigación proporciona una exploración detallada, limpieza de datos, análisis de series temporales del precio de la electricidad.

<img src="images/051a5cb7-768f-4d82-89e1-626b198b7878.jpg" width="600" />

### 📂 Exploración y limpieza:

Los datos constan de **registros horarios** que abarcan desde el año 2015 hasta el 2019. Cada registro se describe mediante múltiples columnas de características relacionadas con la generación de energía y las condiciones meteorológicas en cinco ciudades principales de España: Madrid, Barcelona, Valencia, Sevilla y Bilbao.

> **`'weather_features.csv':`** Contiene información horaria sobre las condiciones meteorológicas (por ejemplo, temperatura, velocidad del viento y humedad).
>>Características:
>>- `time` (Hora del registro): Valor temporal que indica el momento específico del registro.
>>- `temp` (Temperatura en Kelvins): Valor numérico que indica la temperatura.
>>- `temp_min` (Temperatura mínima en Kelvins): Valor numérico que indica la temperatura más baja registrada.
>>- `temp_max` (Temperatura máxima en Kelvins): Valor numérico que indica la temperatura más alta registrada.
>>- `pressure` (Presión atmosférica en hectopascales): Valor numérico que muestra la presión atmosférica.
>>- `humidity` (Humedad relativa en porcentaje): Valor numérico que indica el porcentaje de humedad en el aire.
>>- `wind_speed` (Velocidad del viento en metros por segundo): Valor numérico que muestra la velocidad a la que se desplaza el viento.
>>- `wind_deg` (Dirección del viento en grados): Valor numérico que indica la dirección desde la cual proviene el viento, medida en grados.

> **`'energy_dataset.csv':`** Contiene información horaria sobre la generación de energía. En particular, hay información (en MW) sobre la cantidad de electricidad generada por las distintas fuentes de energía (nuclear, gas fósil y energía eólica dominan la red energética), así como sobre la carga total (demanda de energía) de la red nacional y el precio de la energía (&euro;/MWh). 

>>Características:
>>- `time` (Hora del registro): Valor temporal que indica el momento específico del registro.
>>- `generation biomass` (Generación de biomasa): Valor numérico en MW que indica la cantidad de electricidad generada a partir de biomasa.
>>- `generation fossil brown coal/lignite` (Generación de carbón lignito): Valor numérico en MW que indica la cantidad de electricidad generada a partir de carbón marrón o lignito.
>>- `generation fossil gas` (Generación de gas fósil): Valor numérico en MW que indica la cantidad de electricidad generada a partir de gas fósil.
>>- `generation fossil hard coal` (Generación de carbón duro): Valor numérico en MW que indica la cantidad de electricidad generada a partir de carbón duro.
>>- `generation fossil oil` (Generación de petróleo fósil): Valor numérico en MW que indica la cantidad de electricidad generada a partir de petróleo.
>>- `generation hydro pumped storage consumption` (Consumo de almacenamiento de bombeo hidroeléctrico): Valor numérico en MW que indica la cantidad de electricidad consumida por el almacenamiento de bombeo hidroeléctrico.
>>- `generation hydro run-of-river and poundage` (Generación hidroeléctrica de río y embalse): Valor numérico en MW que indica la cantidad de electricidad generada por tecnologías hidroeléctricas de río y embalse.
>>- `generation hydro water reservoir` (Generación hidroeléctrica de embalse): Valor numérico en MW que indica la cantidad de electricidad generada a partir de embalses.
>>- `generation nuclear` (Generación nuclear): Valor numérico en MW que indica la cantidad de electricidad generada a partir de energía nuclear.
>>- `generation other` (Otras fuentes de generación): Valor numérico en MW que indica la cantidad de electricidad generada a partir de otras fuentes no especificadas.
>>- `generation other renewable` (Generación de otras fuentes renovables): Valor numérico en MW que indica la cantidad de electricidad generada a partir de otras fuentes renovables no especificadas.
>>- `generation solar` (Generación solar): Valor numérico en MW que indica la cantidad de electricidad generada a partir de energía solar.
>>- `generation waste` (Generación a partir de residuos): Valor numérico en MW que indica la cantidad de electricidad generada a partir de residuos.
>>- `generation wind onshore` (Generación eólica en tierra): Valor numérico en MW que indica la cantidad de electricidad generada por turbinas eólicas en tierra.
>>- `total load actual` (Carga total real): Valor numérico en MW que indica la demanda total de electricidad en la red.
>>- `price day ahead` (Precio del día siguiente): Valor numérico en €/MWh que indica el precio acordado de la electricidad para el día siguiente basado en las previsiones.
>>- `price actual` (Precio real o actual): Valor numérico en €/MWh que indica el precio real de la electricidad en el mercado en tiempo real, reflejando las condiciones actuales de oferta y demanda.


---

In [1]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
import warnings

from tensorflow.data import Dataset
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Conv1D, MaxPooling1D, TimeDistributed, Flatten, Dropout, RepeatVector, RNN
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss, ccf
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from math import sqrt

%matplotlib inline

warnings.simplefilter(action='ignore', category=(FutureWarning, UserWarning))
pd.set_option('mode.chained_assignment', 'warn')

plt.style.use('seaborn-v0_8-whitegrid') # print(plt.style.available)
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Load data
weather_columns=['time', 'city_name',
                 'temp', 'temp_min', 'temp_max',
                 'pressure', 'humidity', 'wind_speed', 'wind_deg']

enery_columns = ['time',
                 'generation biomass',
                 'generation fossil brown coal/lignite',
                 'generation fossil gas',
                 'generation fossil hard coal',
                 'generation fossil oil',
                 'generation hydro pumped storage consumption',
                 'generation hydro run-of-river and poundage',
                 'generation hydro water reservoir',
                 'generation nuclear',
                 'generation other',
                 'generation other renewable',
                 'generation solar',
                 'generation waste',
                 'generation wind onshore',
                 'total load actual',
                 #'price day ahead',
                 'price actual']

def parse_utc(x):
    return pd.to_datetime(x, utc=True, infer_datetime_format=True)

df_weather = pd.read_csv('weather_features.csv',
                         parse_dates=['time'],
                         date_parser=parse_utc,
                         usecols=weather_columns,
                         index_col=['time'])

df_energy = pd.read_csv('energy_dataset.csv',
                        parse_dates=['time'],
                        date_parser=parse_utc,
                        usecols=enery_columns,
                        index_col=['time'])

display(df_weather.head(), df_energy.head())

In [ ]:
# Data exploration

# COMPLETE HERE

# 1. Detect gaps in time series

# 2. Convert data types in DataFrame

# 3. Display descriptive statistics

In [ ]:
def plot_series(df=None, column=None, series=pd.Series([]), 
                label=None, ylabel=None, title=None, start=0, end=None):

    sns.set_style("whitegrid")
    sns.set_context("talk")
    fig, ax = plt.subplots(figsize=(30, 12))
    
    ax.set_xlabel('Time', fontsize=18, weight='bold')  
    line_width = 5 
    if column:
        ax.plot(df[column][start:end], label=label, linewidth=line_width)
        ax.set_ylabel(ylabel, fontsize=18, weight='bold')  
    if series.any():
        ax.plot(series, label=label, linewidth=line_width)
        ax.set_ylabel(ylabel, fontsize=18, weight='bold')  
    if label:
        ax.legend(fontsize=18)  
    if title:
        ax.set_title(title, fontsize=26, weight='bold')  
    
    ax.tick_params(axis='x', labelsize=18, labelrotation=0)
    ax.tick_params(axis='y', labelsize=18)
    
    for label in ax.get_xticklabels():
        label.set_weight('bold')
        label.set_color('black')
    
    for label in ax.get_yticklabels():
        label.set_weight('bold')
        label.set_color('black')
    
    ax.grid(True)
    return ax

ax = plot_series(df=df_energy, column='total load actual', ylabel='Total Load (MWh)',
                 title='Actual Total Load (First 2 weeks - Original)', end=24*7*2)
plt.show()

In [ ]:
# DATA IMPUTATION AND CLEANING

# 1. Check missing values

# 2. Check duplicates

# 3. Replace outliers with NaN

# 4. Imputation methods

In [ ]:
# Number of observations per city

In [ ]:
# Merge dataframes

## 📊 Visualizaciones y análisis de series temporales

In [ ]:
# Actual electricity price plot per two weeks
ax = plot_series(df, 'price actual', label='Hourly', ylabel='Actual Price (€/MWh)',
                 start=1 + 24 * 500, end=1 + 24 * 515,
                 title='Actual Hourly Electricity Price (Zoomed - 2 Weeks)')

start_of_first_week = '2016-04-24'
end_of_first_week = '2016-05-01'
ax.axvspan(start_of_first_week, end_of_first_week, facecolor='lightblue', alpha=0.5)

start_of_second_week = '2016-05-01'
end_of_second_week = '2016-05-08'
ax.axvspan(start_of_second_week, end_of_second_week, facecolor='lightgreen', alpha=0.5)

plt.show()

In [ ]:
# Actual electricity price decompose (Y = Trend + Seasonal + Cyclical + Irregular)

# Extract the trend component

# Extract the seasonal component

# Extract the irregular component

## 🔪 Segmentación de Datos Multivariantes en Ventanas

**Concepto Principal:**

La segmentación de datos en ventanas es una técnica esencial cuando se trabaja con series temporales, en particular para modelos de aprendizaje profundo como las redes neuronales. Esta técnica permite transformar un conjunto de datos temporal en segmentos o "ventanas" que facilitan el entrenamiento y la predicción.

La idea es tomar "fotografías" o "ventanas" de datos consecutivos y usarlas como entradas para el modelo. Cada ventana tiene una longitud fija y se mueve a través del conjunto de datos paso a paso.

<table style="width:100%;">
  <tr>
    <td style="text-align:center;"><img src="images/backtesting_refit_fixed_train_size.gif" alt="Backtesting Refit Fixed Train Size"/></td>
    <td style="text-align:center;"><img src="images/Sliding-Windows-Approach-window-size-10-and-step-1.ppm" alt="Sliding Windows Approach"/></td>
  </tr>
</table>

In [ ]:
def multivariate_data(dataset, target, start_index, end_index, windows_size,
                      target_size, step, single_step=False):
# COMPLETE HERE
    return np.array(data), np.array(labels)

La función **multivariate_data** crea conjuntos de datos con ventanas a partir de datos de series temporales, utilizando el método de windowing o ventana deslizante. Para ello, segmenta el conjunto de datos en ventanas superpuestas o no superpuestas y, a continuación, organiza estas ventanas en un formato adecuado.

**Atributos:**
- `dataset`: La matriz de datos de entrada.
- `target`: La matriz de valores objetivo.
- `start_index`: El índice en el que comenzar a extraer datos.
- `end_index`: El índice en el que detener la extracción de datos.
- `history_size`: El tamaño de la ventana histórica de datos a utilizar.
- `target_size`: El número de valores objetivo a predecir.
- `step`: El paso o intervalo entre datos consecutivos en una ventana.
- `single_step`: Un booleano para determinar si predecir un solo paso en el futuro (True) o varios pasos (False).

In [ ]:
# Split the data

### 🤖 Modelos y Capas:

In [ ]:
# Parameters
BATCH_SIZE = 32
BUFFER_SIZE = 1000

# 1. Preparing datasets

# Training dataset
train = Dataset.from_tensor_slices((X_train, y_train))
train = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

# Validation dataset
validation = Dataset.from_tensor_slices((X_val, y_val))
validation = validation.batch(BATCH_SIZE).prefetch(1)

# 2. Model parameters and callbacks

# Input shape for the neural network
input_shape = X_train.shape[-2:]

# Loss function
loss = MeanSquaredError()

# Metrics
metric = [RootMeanSquaredError()]

# Learning rate scheduler
lr_schedule = LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 10))

# Early stopping callback
early_stopping = EarlyStopping(patience=10)

# 3. Post-processing for test data
y_test = y_test.reshape(-1, 1)
y_test_inv = scaler_y.inverse_transform(y_test)

In [ ]:
# plots for model evaluation
def plot_model_rmse_and_loss(history):
    
    train_rmse = history.history['root_mean_squared_error']
    val_rmse = history.history['val_root_mean_squared_error']
    
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    plt.plot(train_rmse, label='Training RMSE')
    plt.plot(val_rmse, label='Validation RMSE')
    plt.legend()
    plt.title('Epochs vs. Training and Validation RMSE')
    
    plt.subplot(1, 2, 2)
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend()
    plt.title('Epochs vs. Training and Validation Loss')
    
    plt.show()
    
def plot_goodness_of_fit(y_true, y_pred):
    
    r2 = r2_score(y_true, y_pred)
    
    fig, ax = plt.subplots()
    
    ax.scatter(y_true, y_pred, alpha=0.5, marker='.', edgecolors='white', linewidths=0.5)
    ax.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()],
            'k--', lw=2, color='black', label='Reference')
    ax.text(0.8, 0.1, f'R² = {r2:.2f}', transform=ax.transAxes, fontsize=12,
            verticalalignment='center', horizontalalignment='center')
    
    ax.set_xlabel('Real Values')
    ax.set_ylabel('Predictions')
    ax.legend()
    
    plt.show()

### RNN (Recurrent Neural Networks):

Las RNN son redes neuronales diseñadas para trabajar con secuencias de datos, donde la información previa podría influir en la salida actual.

A diferencia de las redes neuronales tradicionales, las RNN mantienen una especie de "memoria" al recircular la salida de una capa neuronal a la entrada de la misma. Esto les permite recordar información de entradas anteriores.

![](images/1_SKGAqkVVzT6co-sZ29ze-g.webp)

In [ ]:
# Compile and fit RNN model
clear_session()

model_rnn = Sequential([
# COMPLETE HERE
])

# COMPLETE HERE

In [ ]:
# Evaluate RNN model: performance and visualize results
plot_model_rmse_and_loss(history)

y_pred = model_rnn.predict(X_test)

y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1))
y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

plot_goodness_of_fit(y_test_original, y_pred_original)

### LSTM (Long Short-Term Memory):

Las LSTM son una variante especializada de las Redes Neuronales Recurrentes (RNN). Están diseñadas para recordar patrones a largo plazo y son particularmente útiles para secuencias y series temporales.

A diferencia de las RNN estándar, que pueden tener dificultades para mantener información a lo largo de muchas secuencias debido al "problema de la desaparición del gradiente", las LSTM tienen una estructura de "celda de memoria". Estas celdas permiten que la información sea leída, escrita o borrada, gracias a estructuras llamadas puertas (gates).

![](images/LSTM.jpg)

In [ ]:
# Compile and fit LSTM model
clear_session()

model_lstm = Sequential([
    # COMPLETE HERE
])

# COMPLETE HERE

In [ ]:
# Evaluate LSTM model: performance and visualize results
plot_model_rmse_and_loss(history)

y_pred = model_lstm.predict(X_test)

y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1))
y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

plot_goodness_of_fit(y_test_original, y_pred_original)

### CNN (Convolutional Neural Networks):

Las CNN son redes neuronales que se han popularizado en tareas de procesamiento de imágenes.

En el contexto de series temporales, una convolución funciona como un filtro que se desliza a través de los datos temporales, detectando características específicas en el tiempo. Cada filtro puede estar diseñado para captar una característica temporal diferente, como tendencias, ciclos, o anomalías.
<table style="width:100%;">
  <tr>
    <td style="text-align:center;"><img src="images/bb_copy.gif"/></td>
    <td style="text-align:center;"><img src="images/1_BqNxNytX2Ihp-xGH-XSF8Q.webp"/></td>
  </tr>
</table>

In [ ]:
# compile and fit CNN model
clear_session()

model_cnn = Sequential([
    # COMPLETE HERE
])

# COMPLETE HERE

In [ ]:
# Evaluate CNN model: performance and visualize results
plot_model_rmse_and_loss(history)

y_pred = model_cnn.predict(X_test)

y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1))
y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

plot_goodness_of_fit(y_test_original, y_pred_original)

## 📚 Enlaces de interés

https://www.kaggle.com/code/azminetoushikwasi/time-series-analysis-forecasting

https://www.kaggle.com/code/iamleonie/intro-to-time-series-forecasting